In [2]:
%matplotlib inline

In [4]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import math
import re

# 
df = pd.read_csv("../../X0_raw_data/log2_tab.csv")

df.columns

renamed_col = [x.split("-")[0]+"_"+x.split(" ")[1].split("_")[0]+
 "_"+ x.split("-")[1].split(" ")[0] for x in df.columns if "B6" in x or "TC" in x]

df.columns = ["MID"] + renamed_col + ['pool 1', 'pool 2'] 

df.columns

df.to_csv("../../X0_raw_data/log2_tab_renamed.csv", index = False)

In [31]:
df = pd.read_csv("../../X0_log2_transformed_log2FC/featab_simp_norm_annot_rm_TC-5_log2.csv")

In [32]:
df.columns = ["MID"] + df.columns[1:].tolist()

In [33]:
#2 by 2 comparisons
# TC_Tfh vs B6_Tfh
# TC_Tn vs B6_Tn
# B6_Tfh vs B6_Tn
# TC_Tfh vs TC_Tn

In [46]:
#Set your Exp_id & condition
Exp_id = ""
cond = ["TC.*Tfh","B6.*Tfh"] #"B6.*Tfh","B6.*Tn","TC.*Tfh","TC.*Tn"
cond_name = [x.replace(".*","_") for x in cond]

In [47]:
[x for x in df.columns if Exp_id in x and re.search(cond[0], x)] #using regex

['C1_TC.1_Tfh_010',
 'C1_TC.2_Tfh_011',
 'C1_TC.3_Tfh_012',
 'C2_TC.1_Tfh_018',
 'C2_TC.2_Tfh_019',
 'C2_TC.3_Tfh_020',
 'C2_TC.4_Tfh_021']

In [48]:
res_df = pd.DataFrame(df.iloc[:,0])

for i in range(df.shape[0]):
    try:
        t,p = stats.ttest_ind( [float(x) for x in (df.loc[i,[x for x in df.columns if Exp_id in x and re.search(cond[0], x)]])], 
                              [float(x) for x in (df.loc[i,[x for x in df.columns if Exp_id in x and re.search(cond[1], x)]])] )
        if math.isnan(p) is True:
            res_df.loc[i,"ttest_pval" + Exp_id + "_" + "vs".join(cond_name) ] = 1
            res_df.loc[i,"ttest_tscore_" + Exp_id  + "_" + "vs".join(cond_name)] = 0
        else:
            res_df.loc[i,"ttest_pval" + Exp_id + "_" + "vs".join(cond_name) ] = p
            res_df.loc[i,"ttest_tscore_" + Exp_id  + "_" + "vs".join(cond_name)] = t
    except Exception as e:
        print(e)
        res_df.loc[i,"ttest_pval" + Exp_id + "_" + "vs".join(cond_name) ] = 1
        res_df.loc[i,"ttest_tscore_" + Exp_id  + "_" + "vs".join(cond_name)] = 0

In [49]:
res_df.to_csv("ttest4" + Exp_id + "_" + "vs".join(cond_name) + ".txt", sep='\t', index = None)

-----------

# merge annot df
* This step can be done after BH adjustment of p values

In [50]:
import os

In [51]:
#select the files you would like to merge
file_l = [x for x in os.listdir() if "padj_on_all" in x and ".R" not in x];file_l

['padj_on_all_ttest4_B6_TfhvsB6_Tn.txt',
 'padj_on_all_ttest4_TC_TfhvsB6_Tfh.txt',
 'padj_on_all_ttest4_TC_TfhvsTC_Tn.txt',
 'padj_on_all_ttest4_TC_TnvsB6_Tn.txt']

In [52]:
i = 0
m_df = []
for item in file_l:
    temp = pd.read_table(item)
    if i ==0:
        m_df = temp
    else:
        m_df = pd.merge(m_df,temp, on = "MID")
    i+=1

In [53]:
m_df.to_csv("merged_ttest_padj_on_annot.csv", index=False)

----------

# merge all df
* This step can be done after BH adjustment of p values

In [30]:
import os
import pandas as pd

In [31]:
#select the files you would like to merge
file_l = [x for x in os.listdir() if "padj_on_all" in x and ".R" not in x];file_l

['padj_on_all_ttest4_B6_TFHvsB6_TN.txt',
 'padj_on_all_ttest4_TC_TFHvsB6_TFH.txt',
 'padj_on_all_ttest4_TC_TFHvsTC_TN.txt',
 'padj_on_all_ttest4_TC_TNvsB6_TN.txt']

In [32]:
i = 0
m_df = []
for item in file_l:
    temp = pd.read_table(item)
    if i ==0:
        m_df = temp
    else:
        m_df = pd.merge(m_df,temp, on = "MID")
    i+=1

In [33]:
m_df.to_csv("merged_ttest_padj_on_all.csv", index=False)

-------------

#### Kruskal Wallis test
    KW-test is said to have at least 5 measurements per group :
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kruskal.html
        scipy.stats.kruskal
    U-Whiney test is said to have at least 20 measurements per group :
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html
        scipy.stats.mannwhitneyu

KW-test

In [42]:
#Set your Exp_id & condition
Exp_id = ""
cond = ["PUF1","cobA"]

res_df = df.iloc[:,0:3]
try:
    for i in range(df.shape[0]):
        t,p = stats.kruskal( [float(x) for x in (df.loc[i,[x for x in df.columns if Exp_id in x and cond[0] in x]])], 
                             [float(x) for x in (df.loc[i,[x for x in df.columns if Exp_id in x and cond[1] in x]])],
                             nan_policy = 'omit')
        res_df.loc[i,"htest_pval_" + Exp_id + "_" + "vs".join(cond) ] = p
        res_df.loc[i,"htest_tscore_" + Exp_id  + "_" + "vs".join(cond)] = t
        if isinstance(t,str):
            print("yes")
except Exception as e:
    print(e)        
    
res_df["htest_pval_" + Exp_id + "_" + "vs".join(cond)].replace(np.nan, 1.0 , inplace=True)
res_df["htest_tscore_" + Exp_id + "_" + "vs".join(cond)].replace(np.nan, 1.0 , inplace=True)

    
cols = res_df.columns.tolist()

new_cols = cols[1:] + [cols[0]]
new_cols

res_df_new = res_df[new_cols]

res_df_new.to_csv("htest4mcg_" + Exp_id + "_" + "vs".join(cond) + ".txt", sep='\t', index = None)

U-Whiney test

In [47]:
#Set your Exp_id & condition
Exp_id = "_6_"
cond = ["PUF1","cobA"]

res_df = df.iloc[:,0:3]
try:
    for i in range(df.shape[0]):
        t,p = stats.mannwhitneyu( [float(x) for x in (df.loc[i,[x for x in df.columns if Exp_id in x and cond[0] in x]])], 
                                  [float(x) for x in (df.loc[i,[x for x in df.columns if Exp_id in x and cond[1] in x]])],
                                  use_continuity=True, alternative='two-sided')
        res_df.loc[i,"utest_pval_" + Exp_id + "_" + "vs".join(cond) ] = p
        res_df.loc[i,"utest_tscore_" + Exp_id  + "_" + "vs".join(cond)] = t
        if isinstance(t,str):
            print("yes")
except Exception as e:
    print(e)        
    
res_df["utest_pval_" + Exp_id + "_" + "vs".join(cond)].replace(np.nan, 1.0 , inplace=True)
res_df["utest_tscore_" + Exp_id + "_" + "vs".join(cond)].replace(np.nan, 1.0 , inplace=True)

    
cols = res_df.columns.tolist()

new_cols = cols[1:] + [cols[0]]
new_cols

res_df_new = res_df[new_cols]

res_df_new.to_csv("utest4mcg_" + Exp_id + "_" + "vs".join(cond) + ".txt", sep='\t', index = None)

All numbers are identical in mannwhitneyu
